In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.data.utils import load_graphs
from dgl.nn.pytorch import GraphConv

In [24]:
class CNN_layer(nn.Module):
    def __init__(self, embed_dim, hidden_size, num_filters, filter_size):

        super(CNN_layer, self).__init__()
        self.convs = nn.ModuleList(
            [nn.Conv2d(1, num_filters, (k, embed_dim)) for k in filter_size])
        self.fc = nn.Linear(num_filters * len(filter_size), hidden_size)
        self.value = torch.tensor(1.0, dtype=float, requires_grad=True)

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, x):
        out = x.unsqueeze(1)
        out = torch.cat([self.conv_and_pool(out, conv) for conv in self.convs],
                        1)
        out = self.fc(out)
        out = F.tanh(out)
        return out


In [25]:
embed_dim = 300
hidden_size = 128
num_filters = 32
filter_size = range(2, 6)


In [26]:
cnn = CNN_layer(embed_dim, hidden_size, num_filters, filter_size)


In [27]:
x = torch.randn((1119, 1000, 300))


In [28]:
out = cnn(x)

/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [29]:
out.shape

torch.Size([1119, 128])

In [30]:
class Graph_model(nn.Module):
    def __init__(self, graph_type, hidden_size, num_classes):
        super(Graph_model, self).__init__()
        self.graph_type = graph_type
        self.g = self.graph_make()
        self.conv1 = GraphConv(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)
        

    def graph_make(self):
        glist, lable = load_graphs('/new_python_for_gnn/毕设code/data/graph.bin')
        return glist[self.graph_type]

    def forward(self, x):
        out = self.conv1(self.g, x)
        out = F.tanh(out)
        out = self.fc(out)
        return out

In [31]:
graph_conv = Graph_model(0,128,2)

In [33]:
graph_conv(out).shape

/home/nsc/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


torch.Size([1119, 2])